In [0]:
display(dbutils.fs.ls("abfss://raw-data@dyrczdata.dfs.core.windows.net/"))

path,name,size,modificationTime
abfss://raw-data@dyrczdata.dfs.core.windows.net/Superstore.csv,Superstore.csv,2297016,1770980582000
abfss://raw-data@dyrczdata.dfs.core.windows.net/rfm_results_delta/,rfm_results_delta/,0,1770988038000


In [0]:
path = "abfss://raw-data@dyrczdata.dfs.core.windows.net/Superstore.csv"

df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(path)

print("GRATULACJE! Dane wczytane poprawnie.")

display(df.limit(10))

from pyspark.sql.functions import sum, round, col

df_report = df.groupBy("Category") \
    .agg(round(sum("Profit"), 2).alias("Total_Profit")) \
    .orderBy(col("Total_Profit").desc())

print("Raport zysków według kategorii:")
display(df_report)

GRATULACJE! Dane wczytane poprawnie.


Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136
2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582
3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714
4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.031
5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.2,2.5164
6,CA-2015-115812,2015-06-09,2015-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,FUR-FU-10001487,Furniture,Furnishings,"Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood",48.86,7,0,14.1694
7,CA-2015-115812,2015-06-09,2015-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.28,4,0,1.9656
8,CA-2015-115812,2015-06-09,2015-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.152,6,0.2,90.7152
9,CA-2015-115812,2015-06-09,2015-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by Samsill,18.504,3,0.2,5.7825
10,CA-2015-115812,2015-06-09,2015-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9,5,0,34.47


Raport zysków według kategorii:


Category,Total_Profit
Technology,145388.3
Office Supplies,120632.88
Furniture,19686.43


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import col, to_date, trim, when

path = "abfss://raw-data@dyrczdata.dfs.core.windows.net/Superstore.csv"
df_raw = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(path)

df_clean = df_raw.select([col(c).alias(c.replace(' ', '_').lower()) for c in df_raw.columns]) \
    .withColumn("order_date", to_date(col("order_date"), "M/d/yyyy")) \
    .withColumn("ship_date", to_date(col("ship_date"), "M/d/yyyy")) \
    .dropDuplicates()

df_clean = df_clean.fillna({'postal_code': 0, 'region': 'Unknown'})

print("Dane wyczyszczone. Liczba rekordów po usunięciu duplikatów:", df_clean.count())
display(df_clean.limit(5))

Dane wyczyszczone. Liczba rekordów po usunięciu duplikatów: 9994


row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
3349,CA-2018-154732,2018-11-05,2018-11-07,First Class,AH-10195,Alan Haines,Corporate,United States,Chicago,Illinois,60623,Central,OFF-BI-10000474,Office Supplies,Binders,Avery Recycled Flexi-View Covers for Binding Systems,16.03,5,0.8,-25.648
3498,CA-2017-107328,2017-08-08,2017-08-15,Standard Class,CA-12055,Cathy Armstrong,Home Office,United States,Los Angeles,California,90036,West,OFF-AP-10004487,Office Supplies,Appliances,Kensington 4 Outlet MasterPiece Compact Power Control Center,487.92,6,0,136.6176
764,CA-2015-162775,2015-01-13,2015-01-15,Second Class,CS-12250,Chris Selesnick,Corporate,United States,Bossier City,Louisiana,71111,South,OFF-EN-10001990,Office Supplies,Envelopes,Staple envelope,11.36,2,0,5.3392
1550,CA-2015-134278,2015-07-06,2015-07-08,First Class,EP-13915,Emily Phan,Consumer,United States,New York City,New York,10011,East,TEC-CO-10001046,Technology,Copiers,Canon Imageclass D680 Copier / Fax,559.992,1,0.2,174.9975
2518,CA-2018-123134,2018-05-02,2018-05-07,Standard Class,DW-13585,Dorothy Wardle,Corporate,United States,Westfield,New Jersey,7090,East,FUR-FU-10003975,Furniture,Furnishings,Eldon Advantage Chair Mats for Low to Medium Pile Carpets,129.93,3,0,12.993


In [0]:
from pyspark.sql.functions import *

df_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .load("abfss://raw-data@dyrczdata.dfs.core.windows.net/Superstore.csv")

df_silver = df_raw.select(
    *[col(c).alias(c.replace(' ', '_').lower()) for c in df_raw.columns]
).withColumn(
    "order_date_clean", 
    coalesce(
        to_date(col("order_date"), "yyyy-MM-dd"),
        to_date(col("order_date"), "M/d/yyyy")
    )
).withColumn(
    "sales", regexp_replace(col("sales"), "[^0-9.]", "").cast("double")
).withColumn(
    "profit", regexp_replace(col("profit"), "[^0-9.-]", "").cast("double")
)

df_silver = df_silver.filter(col("order_date_clean").isNotNull() & col("sales").isNotNull())

print(f"Dane oczyszczone. Pozostało rekordów: {df_silver.count()}")
display(df_silver.limit(5))

Dane oczyszczone. Pozostało rekordów: 9994


row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit,order_date_clean
1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136,2017-11-08
2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582,2017-11-08
3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714,2017-06-12
4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.031,2016-10-11
5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.2,2.5164,2016-10-11


In [0]:
df_shipping = df_silver.withColumn(
    "ship_date_clean", coalesce(to_date(col("ship_date"), "yyyy-MM-dd"), to_date(col("ship_date"), "M/d/yyyy"))
).withColumn(
    "days_to_ship", datediff(col("ship_date_clean"), col("order_date_clean"))
)

anomalies_shipping = df_shipping.filter((col("days_to_ship") < 0) | (col("days_to_ship") > 15))

print("Wykryto anomalie w datach wysyłki:")
display(anomalies_shipping.select("order_id", "order_date", "ship_date", "days_to_ship"))

Wykryto anomalie w datach wysyłki:


order_id,order_date,ship_date,days_to_ship


In [0]:
stats = df_silver.select(mean("sales").alias("avg"), stddev("sales").alias("std")).collect()
avg_val, std_val = stats[0]["avg"], stats[0]["std"]

df_outliers = df_silver.filter(col("sales") > (avg_val + 3 * std_val))

print(f"Znaleziono {df_outliers.count()} transakcji o ekstremalnie wysokiej wartości:")
display(df_outliers.orderBy(col("sales").desc()))

Znaleziono 127 transakcji o ekstremalnie wysokiej wartości:


row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit,order_date_clean
2698,CA-2015-145317,2015-03-18,2015-03-23,Standard Class,SM-20320,Sean Miller,Home Office,United States,Jacksonville,Florida,32216,South,TEC-MA-10002412,Technology,Machines,Cisco TelePresence System EX90 Videoconferencing Unit,22638.48,6,0.5,-1811.0784,2015-03-18
6827,CA-2017-118689,2017-10-02,2017-10-09,Standard Class,TC-20980,Tamara Chand,Corporate,United States,Lafayette,Indiana,47905,Central,TEC-CO-10004722,Technology,Copiers,Canon imageCLASS 2200 Advanced Copier,17499.95,5,0,8399.976,2017-10-02
8154,CA-2018-140151,2018-03-23,2018-03-25,First Class,RB-19360,Raymond Buch,Consumer,United States,Seattle,Washington,98115,West,TEC-CO-10004722,Technology,Copiers,Canon imageCLASS 2200 Advanced Copier,13999.96,4,0,6719.9808,2018-03-23
2624,CA-2018-127180,2018-10-22,2018-10-24,First Class,TA-21385,Tom Ashbrook,Home Office,United States,New York City,New York,10024,East,TEC-CO-10004722,Technology,Copiers,Canon imageCLASS 2200 Advanced Copier,11199.968,4,0.2,3919.9888,2018-10-22
4191,CA-2018-166709,2018-11-17,2018-11-22,Standard Class,HL-15040,Hunter Lopez,Consumer,United States,Newark,Delaware,19711,East,TEC-CO-10004722,Technology,Copiers,Canon imageCLASS 2200 Advanced Copier,10499.97,3,0,5039.9856,2018-11-17
9040,CA-2017-117121,2017-12-17,2017-12-21,Standard Class,AB-10105,Adrian Barton,Consumer,United States,Detroit,Michigan,48205,Central,OFF-BI-10000545,Office Supplies,Binders,GBC Ibimaster 500 Manual ProClick Binding System,9892.74,13,0,4946.37,2017-12-17
4099,CA-2015-116904,2015-09-23,2015-09-28,Standard Class,SC-20095,Sanjit Chand,Consumer,United States,Minneapolis,Minnesota,55407,Central,OFF-BI-10001120,Office Supplies,Binders,Ibico EPK-21 Electric Binding System,9449.95,5,0,4630.4755,2015-09-23
4278,US-2017-107440,2017-04-16,2017-04-20,Standard Class,BS-11365,Bill Shonely,Corporate,United States,Lakewood,New Jersey,8701,East,TEC-MA-10001047,Technology,Machines,"3D Systems Cube Printer, 2nd Generation, Magenta",9099.93,7,0,2365.9818,2017-04-16
8489,CA-2017-158841,2017-02-02,2017-02-04,Second Class,SE-20110,Sanjit Engle,Consumer,United States,Arlington,Virginia,22204,South,TEC-MA-10001127,Technology,Machines,"HP Designjet T520 Inkjet Large Format Printer - 24"" Color",8749.95,5,0,2799.984,2017-02-02
6426,CA-2017-143714,2017-05-23,2017-05-27,Standard Class,CC-12370,Christopher Conant,Consumer,United States,Philadelphia,Pennsylvania,19120,East,TEC-CO-10004722,Technology,Copiers,Canon imageCLASS 2200 Advanced Copier,8399.976,4,0.4,1119.9968,2017-05-23


In [0]:

df_toxic_products = df_silver.groupBy("product_name", "category") \
    .agg(
        sum("sales").alias("total_sales"),
        sum("profit").alias("total_profit"),
        avg("discount").alias("avg_discount")
    ) \
    .filter(col("total_profit") < 0) \
    .orderBy(col("total_profit"))

print("Produkty 'Toksyczne' (największe straty):")
display(df_toxic_products.limit(10))

Produkty 'Toksyczne' (największe straty):


product_name,category,total_sales,total_profit,avg_discount
Cubify CubeX 3D Printer Double Head Print,Technology,11099.963,-8879.9704,0.5333333333333333
Lexmark MX611dhe Monochrome Laser Printer,Technology,16829.901,-4589.973,0.4
Cubify CubeX 3D Printer Triple Head Print,Technology,7999.98,-3839.9904,0.5
Chromcraft Bull-Nose Wood Oval Conference Tables & Bases,Furniture,9917.64,-2876.1156,0.28
Bush Advantage Collection Racetrack Conference Table,Furniture,9544.725,-1934.3976000000002,0.35000000000000003
GBC DocuBind P400 Electric Binding System,Office Supplies,17965.068,-1878.1662000000003,0.45
Cisco TelePresence System EX90 Videoconferencing Unit,Technology,22638.48,-1811.0784,0.5
Martin Yale Chadless Opener Electric Letter Opener,Office Supplies,16656.199999999997,-1299.1836,0.10000000000000002
Balt Solid Wood Round Tables,Furniture,6518.754000000001,-1201.0581,0.2
BoxOffice By Design Rectangular and Half-Moon Meeting Room Tables,Furniture,1706.25,-1148.4375,0.48333333333333334


In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import *

current_date = df_silver.select(max("order_date_clean")).collect()[0][0]

rfm_base = df_silver.groupBy("customer_id", "customer_name").agg(
    datediff(lit(current_date), max("order_date_clean")).alias("recency"),
    countDistinct("order_id").alias("frequency"),
    round(sum("sales"), 2).alias("monetary")
)


def get_rfm_scores(df):
    w_r = Window.orderBy(col("recency").desc()) 
    w_f = Window.orderBy(col("frequency").asc())
    w_m = Window.orderBy(col("monetary").asc())
    
    return df.withColumn("r_score", ntile(4).over(w_r)) \
             .withColumn("f_score", ntile(4).over(w_f)) \
             .withColumn("m_score", ntile(4).over(w_m))

rfm_scored = get_rfm_scores(rfm_base)

rfm_final = rfm_scored.withColumn("total_rfm", col("r_score") + col("f_score") + col("m_score")) \
    .withColumn("segment", 
        when(col("total_rfm") >= 11, "Mistrzowie (Champions)")
        .when(col("total_rfm") >= 9, "Lojalni Klienci")
        .when(col("total_rfm") >= 7, "Obiecujący")
        .when(col("total_rfm") >= 5, "Wymagający uwagi")
        .otherwise("Klienci traceni / Ryzykowni")
    )

print("Segmentacja RFM zakończona sukcesem!")
display(rfm_final.orderBy(col("total_rfm").desc()))

Segmentacja RFM zakończona sukcesem!


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


customer_id,customer_name,recency,frequency,monetary,r_score,f_score,m_score,total_rfm,segment
MM-17920,Michael Moore,7,11,3794.08,4,4,4,12,Mistrzowie (Champions)
AT-10735,Annie Thurman,13,10,3831.86,4,4,4,12,Mistrzowie (Champions)
CC-12220,Chris Cortes,20,12,3913.42,4,4,4,12,Mistrzowie (Champions)
FP-14320,Frank Preis,23,8,4046.75,4,4,4,12,Mistrzowie (Champions)
DM-13345,Denise Monton,22,8,4074.47,4,4,4,12,Mistrzowie (Champions)
TC-21295,Toby Carlisle,27,8,4266.81,4,4,4,12,Mistrzowie (Champions)
ND-18370,Natalie DeCherney,27,9,4326.14,4,4,4,12,Mistrzowie (Champions)
GZ-14470,Gary Zandusky,7,9,4355.15,4,4,4,12,Mistrzowie (Champions)
AI-10855,Arianne Irving,13,10,4375.79,4,4,4,12,Mistrzowie (Champions)
TP-21130,Theone Pippenger,8,9,4454.06,4,4,4,12,Mistrzowie (Champions)


In [0]:
segment_summary = rfm_final.groupBy("segment").agg(
    count("*").alias("customer_count"),
    round(avg("monetary"), 2).alias("avg_spend"),
    round(avg("recency"), 1).alias("avg_days_since_order")
).orderBy(col("customer_count").desc())

display(segment_summary)

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


segment,customer_count,avg_spend,avg_days_since_order
Lojalni Klienci,209,3911.61,68.1
Wymagający uwagi,183,1543.71,177.0
Obiecujący,181,3040.95,111.6
Klienci traceni / Ryzykowni,115,802.51,410.0
Mistrzowie (Champions),105,5280.72,23.0


In [0]:
toxic_vips = rfm_final.join(
    df_silver.groupBy("customer_id").agg(
        round(sum("profit"), 2).alias("total_actual_profit"),
        round(avg("discount"), 2).alias("avg_applied_discount"),
        count("order_id").alias("total_orders_count")
    ),
    "customer_id"
)

toxic_vips_analysis = toxic_vips.filter(
    (col("segment").isin("Mistrzowie (Champions)", "Lojalni Klienci")) & 
    (col("total_actual_profit") < 0)
).select(
    "customer_name", 
    "segment", 
    "monetary", 
    "total_actual_profit", 
    "avg_applied_discount",
    "total_orders_count"
).orderBy("total_actual_profit")

print("RAPORT: Klienci VIP generujący straty (Toksyczni VIP-owie):")
display(toxic_vips_analysis)

RAPORT: Klienci VIP generujący straty (Toksyczni VIP-owie):


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


customer_name,segment,monetary,total_actual_profit,avg_applied_discount,total_orders_count
Cindy Stewart,Lojalni Klienci,5690.05,-6626.39,0.2,9
Luke Foster,Mistrzowie (Champions),3930.51,-3583.98,0.32,16
Sharelle Roach,Lojalni Klienci,3233.48,-3333.91,0.37,9
Henry Goldwyn,Mistrzowie (Champions),3247.64,-2797.96,0.17,17
Sean Braxton,Lojalni Klienci,8057.89,-2082.75,0.24,17
Christine Phan,Lojalni Klienci,5888.28,-1850.3,0.21,15
Natalie Fritzler,Lojalni Klienci,8322.83,-1695.97,0.25,14
Tracy Blumstein,Lojalni Klienci,4737.49,-1603.05,0.27,20
Dan Campbell,Lojalni Klienci,3336.17,-1441.63,0.21,18
David Bremer,Lojalni Klienci,2973.09,-1421.77,0.14,14


Databricks visualization. Run in Databricks to view.

In [0]:
output_path = "abfss://raw-data@dyrczdata.dfs.core.windows.net/toxic_vips_report"

toxic_vips_analysis.write.format("delta").mode("overwrite").save(output_path)

print(f"Raport o Toksycznych VIP-ach zapisany w: {output_path}")

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


Raport o Toksycznych VIP-ach zapisany w: abfss://raw-data@dyrczdata.dfs.core.windows.net/toxic_vips_report


In [0]:
output_path = "abfss://raw-data@dyrczdata.dfs.core.windows.net/clean_silver_data"

df_silver.write.format("delta").mode("overwrite").save(output_path)

print(f"Wyczyszczone dane (Silver) zapisane w: {output_path}")

✅ Wyczyszczone dane (Silver) zapisane w: abfss://raw-data@dyrczdata.dfs.core.windows.net/clean_silver_data


In [0]:
display(dbutils.fs.ls("abfss://raw-data@dyrczdata.dfs.core.windows.net/"))

path,name,size,modificationTime
abfss://raw-data@dyrczdata.dfs.core.windows.net/Superstore.csv,Superstore.csv,2297016,1770980582000
abfss://raw-data@dyrczdata.dfs.core.windows.net/clean_silver_data/,clean_silver_data/,0,1771011148000
abfss://raw-data@dyrczdata.dfs.core.windows.net/rfm_results_delta/,rfm_results_delta/,0,1770988038000
abfss://raw-data@dyrczdata.dfs.core.windows.net/toxic_vips_report/,toxic_vips_report/,0,1771011091000
